In [ ]:
import cartopy.crs as ccrs
import matplotlib.pylab as plt
import cartopy.feature as cfeature
import numpy as np
import xarray as xr
import cmaps
from  scipy  import stats
import matplotlib as mpl
from eofs.standard import Eof
import sys
sys.path.append('/public/home/songqh/Documents/python_packages/')
from sqh_toolbox import plot_figure
import warnings
warnings.filterwarnings("ignore") 
plt.rcParams["font.family"] = "Liberation Sans"
plt.rcParams["font.weight"] = "regular"

In [ ]:
file1 = xr.open_dataset("/public/home/songqh/my_data/new_ERA5/monthly/ERA5_precipitation_1940-2024.nc")
precip_index = file1['tp'].loc[file1.time.dt.year.isin(np.arange(1979,2025,1))&file1.time.dt.month.isin([6,7,8]),28:10,70:100].groupby('time.year').mean('time').rename({'year':'time'}).mean(('longitude','latitude'))*1000
precip_index = (precip_index - np.mean(precip_index,axis=0))/np.std(precip_index)

In [ ]:
file0 = xr.open_dataset("/public/home/songqh/my_data/ice_cover/HadISST_ice.nc")
file0.coords['longitude'] = np.mod(file0['longitude'], 360)
file0 = file0.reindex({ 'longitude' : np.sort(file0['longitude'])})
sic = file0['sic'].loc[file0.time.dt.year.isin(np.arange(1979,2025,1))
                        & file0.time.dt.month.isin([6,7,8]),-30:-90].groupby('time.year').mean('time')
sic_mean = sic.mean('year')
sic_ano = sic - sic_mean
sic_std = np.std(sic,axis=0)
lon1 = file0['longitude'].loc[120:350]
lat1 = file0['latitude'].loc[-40:-90]
sic_zone = sic_ano.loc[:,-40:-90,120:350].values
lat1 = np.array(lat1)
coslat = np.cos(np.deg2rad(lat1))
wgts = np.sqrt(coslat)[..., np.newaxis]
solver = Eof(sic_zone, weights=wgts)
eof =solver.eofsAsCorrelation(neofs=3)
pc = solver.pcs(npcs=3, pcscaling=1)
var = solver.varianceFraction()
print(var[:3])
pc1 = pc[:,0]
pc2 = pc[:,1]
pc3 = pc[:,2]
eof1 = eof[0,:,:]
eof2 = eof[1,:,:]
eof3 = eof[2,:,:]

In [ ]:
r,p = stats.pearsonr(pc1,precip_index[:])
print("r:",r,"p:",p)

In [ ]:
def get_cmap_pr(n_colors: int, newcmap , method: int = None) -> list:
    index = list(range(1, n_colors + 1))
    color_list = [newcmap(i / n_colors) for i in index]
    if method == 1:
        color_list[0] = [1., 1., 1.]  
    elif method == 2:
        mid_index = len(color_list) // 2
        color_list[mid_index] = [1., 1., 1.]  
        color_list[mid_index - 1] = [1., 1., 1.]
    return color_list
new_cmocean_balance_r = mpl.colors.ListedColormap(get_cmap_pr(20,cmaps.cmocean_balance_r,method=2))

In [ ]:
fig = plt.figure(figsize=(12,12))
map_proj = ccrs.LambertConformal(
    central_longitude=230, standard_parallels=(-60, -50))
ax1 = fig.add_subplot(211,projection =map_proj)
ax1.set_extent([120, 340, -90, -55], crs=ccrs.PlateCarree())
ax1.spines['geo'].set_linewidth(1)
ax1.spines['geo'].set_zorder(4)
ax1.spines['geo'].set_edgecolor('black')
cf1 = ax1.contourf(lon1,lat1,-eof1, zorder=1,levels=np.linspace(-1,1,21),  
                   extend = 'both',transform=ccrs.PlateCarree(), cmap=new_cmocean_balance_r)
ax1.add_feature(cfeature.COASTLINE.with_scale('110m'),color='k',linewidth=1,zorder=4)
ax1.add_feature(cfeature.LAND.with_scale('110m'),color='lightgray',zorder=3)
ax1.set_title("JJA SIC EOF1 (var=28%)",fontsize=20,loc='center')
ax1.set_title("a",fontsize=24,weight='bold',loc='left')
gl = ax1.gridlines(draw_labels=False, linestyle='--', color='gray', alpha=0.7, linewidth=0.5,zorder=4)
pos = fig.add_axes([0.87,0.58,0.015,0.26])
cb = fig.colorbar(cf1,orientation='vertical',cax=pos)
cb.ax.set_colorbar(minor='off')

ax2 = fig.add_axes([0.168,0.22,0.69,0.25])
ax2.plot(np.arange(1979,2025),precip_index,c="#d76364",lw=2.5,markersize=8,label='ERA5-rainfall',zorder=1)
ax2.plot(np.arange(1979,2025),pc1,zorder=2,color='#2F7FC1',lw=2.5,markersize=8,label='SIC-PC1')
ax2.set_detail()
ax2.axhline(0,c='k',lw=0.5,linestyle='--')
ax2.set_ylim(-3.5,3.5)
ax2.set_xlim(1978,2025)
ax2.set_title("JJA rainfall & SIC-PC1",fontsize=20)
ax2.set_title("b",fontsize=24,weight='bold',loc='left')
ax2.text(2003,-2.5,"R=0.10 (P>0.1)",fontsize=18)
ax2.legend(frameon=False,fontsize=18,ncol=2,handlelength=1.15,handletextpad=0.3,columnspacing=1.5,labelspacing=0.5,loc='upper left')

fig.savefig("/public/home/songqh/project/ISM_impact_Antarctica/figures/response_figures/support-2.jpeg",dpi=600, bbox_inches='tight')